# Explore quality control statistics for election audits


Given reported results, and evidence from a random sample of ballots, does the sample plausibly come from the reported results?

If not, might there be problems with the sample, the interpretations, or the original tabulation?

Follow the approach of the LRT test statistic described at https://www.biostat.wisc.edu/~kbroman/teaching/labstat/fourth/notes02.pdf


In [1]:
import io
import math
import numpy as np
import pandas as pd
from scipy.stats import percentileofscore

In [2]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=0)

Data from audit of Montgomery OH 2020 primary

In [3]:
results = """	Candidates	Results	Round 1	Total	Required
0	Bennet	51	0	0	
1	Biden	29011	118	118	107
2	Bloomberg	702	4	4	
3	Buttigieg	525	2	2	
4	Gabbard	137	0	0	
5	Klobuchar	406	1	1	
6	Patrick	27	0	0	
7	Sanders	5713	20	20	
8	Steyer	62	0	0	
9	Warren	1118	4	4	
10	Write_Ins	122	1	1	
"""

In [4]:
s = io.StringIO(results)

In [5]:
rdf = pd.read_csv(s, delimiter='	')

In [6]:
rdf

,Unnamed: 0,Candidates,Results,Round 1,Total,Required
0,0,Bennet,51,0,0,NaN
1,1,Biden,29011,118,118,107.0
2,2,Bloomberg,702,4,4,NaN
3,3,Buttigieg,525,2,2,NaN
4,4,Gabbard,137,0,0,NaN
5,5,Klobuchar,406,1,1,NaN
6,6,Patrick,27,0,0,NaN
7,7,Sanders,5713,20,20,NaN
8,8,Steyer,62,0,0,NaN
9,9,Warren,1118,4,4,NaN


In [7]:
reported = rdf.Results.to_numpy()

In [8]:
reported_prob = reported / sum(reported)

In [9]:
reported_prob

array([1.34657021e-03, 7.65987221e-01, 1.85351428e-02, 1.38617521e-02,
       3.61725722e-03, 1.07197550e-02, 7.12890109e-04, 1.50842266e-01,
       1.63700692e-03, 2.95189312e-02, 3.22120716e-03])

In [10]:
sample = [0, 118, 4, 2, 0, 1, 0, 20, 0, 4, 1]

In [11]:
sample_size = sum(sample)

In [12]:
sample_size

150

maximum-likelihood sample

In [13]:
ml_sample = reported_prob * sample_size

In [14]:
ml_sample

array([2.01985531e-01, 1.14898083e+02, 2.78027143e+00, 2.07926282e+00,
       5.42588583e-01, 1.60796325e+00, 1.06933516e-01, 2.26263400e+01,
       2.45551038e-01, 4.42783968e+00, 4.83181074e-01])

Look at differences between original sample and maximum-likelihood sample

In [15]:
[round(s-m, 2) for s,m in zip(sample, ml_sample)]

[-0.2, 3.1, 1.22, -0.08, -0.54, -0.61, -0.11, -2.63, -0.25, -0.43, 0.52]

Generate a bunch of samples from the reported distribution

In [16]:
def lrt_stat(counts, expected_counts):
    """Likelihood ratio test statistic.
    counts (list(number)):          observed counts
    expected_counts (list(float)):  expected counts

    Return a statistic measuring how likely is it that the observed counts 
    comes from the multinomial distribution exemplified by expected_counts.
    The sum of counts and expected counts should be the same.

    Cf. chi squared, but works for smaller numbers?
    from https://www.biostat.wisc.edu/~kbroman/teaching/labstat/fourth/notes02.pdf

    >>> lrt_stat([35, 43, 22], [25.0, 50.0, 25.0])
    4.957619699875772
    >>> lrt_stat([0, 43, 22], [25.0, 50.0, 25.0])
    -18.59543686360913

    TODO: is substituting 0.5 an ok way to handle 0 counts? Need to avoid domain error in log.
    """

    return 2 * sum(ni * math.log(max(ni,0.5) / ni0) for ni, ni0 in zip(counts, expected_counts))

In [17]:
def lrt_pres(data):
    "Run lrt_stat on data"
    return lrt_stat(data, ml_sample)

# Generate distribution of given test statistic via simulation

In [18]:
trials = 100000
# trials = 1000000 # 1M takes a few minutes

In [19]:
trialset = np.random.multinomial(sample_size, reported_prob, size=trials)

In [20]:
test_stats_detail = [(round(lrt_stat(trial, reported_prob * sample_size), 3), trial) for trial in trialset]

In [21]:
test_stats = sorted([v[0] for v in test_stats_detail])

Pick some extreme probabilities for the quantiles

In [22]:
probs = [0.0] + [10**n for n in range(-6, 0)] + [(1-10**n) for n in range(-1, -7, -1)] + [1.0]

In [23]:
quantiles = np.quantile(test_stats, probs)

In [24]:
list(zip(probs, quantiles))

[(0.0, 2.007),
 (1e-06, 2.011799952),
 (1e-05, 2.05499952),
 (0.0001, 2.173),
 (0.001, 2.603),
 (0.01, 3.368),
 (0.1, 5.03),
 (0.9, 14.624),
 (0.99, 20.947039999999976),
 (0.999, 25.948035000000136),
 (0.9999, 31.02901410000067),
 (0.99999, 39.92606884002331),
 (0.999999, 46.121606884012344),
 (1.0, 46.81)]

The original Montgomery sample is rather close to the expected sample - closer than all but 2.3% of random samples.

In [25]:
lrt_sample = lrt_stat(sample, reported_prob * sample_size)

In [26]:
percentileofscore(test_stats, lrt_sample)

2.308

# Explore what outliers would look like

In [27]:
# Incorrect data entry of contest, with one extra candidate, shifting e.g. Sanders votes to 
typosample = [0, 118, 4, 2, 0, 2, 0, 0, 26, 0, 4]

In [28]:
typosample

[0, 118, 4, 2, 0, 2, 0, 0, 26, 0, 4]

In [29]:
sample

[0, 118, 4, 2, 0, 1, 0, 20, 0, 4, 1]

In [30]:
lrt_pres(typosample)

269.26535915406293

In [31]:
def perturb(sample, count, index=0):
    "perturb the 'index' dimension of the sample by count and return the LRT stat, p-value and perturbed sample"

    dims = len(sample)
    perturbed = np.array(sample)
    perturbed[index] += count
    lrt = lrt_pres(perturbed)
    return (round(lrt, 3), round(100.0-percentileofscore(test_stats, lrt), 3), str(perturbed))

Adding more than 2 or 3 to the sample count for first candidate might raise a flag (p-value of 0.014 or 0.00064)

In [32]:
[(i, perturb(sample, i, 0)) for i in range(-4, 10)]

[(-4, (-3.453, 100.0, '[ -4 118   4   2   0   1   0  20   0   4   1]')),
 (-3, (-1.641, 100.0, '[ -3 118   4   2   0   1   0  20   0   4   1]')),
 (-2, (0.172, 100.0, '[ -2 118   4   2   0   1   0  20   0   4   1]')),
 (-1, (1.985, 100.0, '[ -1 118   4   2   0   1   0  20   0   4   1]')),
 (0, (3.798, 97.692, '[  0 118   4   2   0   1   0  20   0   4   1]')),
 (1, (6.997, 70.936, '[  1 118   4   2   0   1   0  20   0   4   1]')),
 (2, (12.969, 17.063, '[  2 118   4   2   0   1   0  20   0   4   1]')),
 (3, (19.987, 1.432, '[  3 118   4   2   0   1   0  20   0   4   1]')),
 (4, (27.685, 0.042, '[  4 118   4   2   0   1   0  20   0   4   1]')),
 (5, (35.888, 0.004, '[  5 118   4   2   0   1   0  20   0   4   1]')),
 (6, (44.494, 0.001, '[  6 118   4   2   0   1   0  20   0   4   1]')),
 (7, (53.434, 0.0, '[  7 118   4   2   0   1   0  20   0   4   1]')),
 (8, (62.662, 0.0, '[  8 118   4   2   0   1   0  20   0   4   1]')),
 (9, (72.14, 0.0, '[  9 118   4   2   0   1   0  20   0   4   1]'

Adding even 6 votes to Biden's count doesn't look very unusual

In [33]:
[(i, perturb(sample, i, 1)) for i in range(-4, 15)]

[(-4, (-4.278, 100.0, '[  0 114   4   2   0   1   0  20   0   4   1]')),
 (-3, (-2.285, 100.0, '[  0 115   4   2   0   1   0  20   0   4   1]')),
 (-2, (-0.275, 100.0, '[  0 116   4   2   0   1   0  20   0   4   1]')),
 (-1, (1.753, 100.0, '[  0 117   4   2   0   1   0  20   0   4   1]')),
 (0, (3.798, 97.692, '[  0 118   4   2   0   1   0  20   0   4   1]')),
 (1, (5.86, 82.604, '[  0 119   4   2   0   1   0  20   0   4   1]')),
 (2, (7.938, 60.467, '[  0 120   4   2   0   1   0  20   0   4   1]')),
 (3, (10.033, 38.316, '[  0 121   4   2   0   1   0  20   0   4   1]')),
 (4, (12.145, 21.724, '[  0 122   4   2   0   1   0  20   0   4   1]')),
 (5, (14.273, 11.247, '[  0 123   4   2   0   1   0  20   0   4   1]')),
 (6, (16.418, 5.365, '[  0 124   4   2   0   1   0  20   0   4   1]')),
 (7, (18.578, 2.415, '[  0 125   4   2   0   1   0  20   0   4   1]')),
 (8, (20.755, 1.074, '[  0 126   4   2   0   1   0  20   0   4   1]')),
 (9, (22.947, 0.432, '[  0 127   4   2   0   1   0  20   0 

Sanders' counts aren't much more sensitive than Biden's, with this test, in this range.

In [34]:
[(i, perturb(sample, i, 7)) for i in range(-4, 15)]

[(-4, (-2.356, 100.0, '[  0 118   4   2   0   1   0  16   0   4   1]')),
 (-3, (-0.987, 100.0, '[  0 118   4   2   0   1   0  17   0   4   1]')),
 (-2, (0.498, 100.0, '[  0 118   4   2   0   1   0  18   0   4   1]')),
 (-1, (2.096, 99.998, '[  0 118   4   2   0   1   0  19   0   4   1]')),
 (0, (3.798, 97.692, '[  0 118   4   2   0   1   0  20   0   4   1]')),
 (1, (5.6, 85.061, '[  0 118   4   2   0   1   0  21   0   4   1]')),
 (2, (7.498, 65.417, '[  0 118   4   2   0   1   0  22   0   4   1]')),
 (3, (9.487, 43.744, '[  0 118   4   2   0   1   0  23   0   4   1]')),
 (4, (11.562, 25.617, '[  0 118   4   2   0   1   0  24   0   4   1]')),
 (5, (13.721, 13.477, '[  0 118   4   2   0   1   0  25   0   4   1]')),
 (6, (15.96, 6.294, '[  0 118   4   2   0   1   0  26   0   4   1]')),
 (7, (18.276, 2.726, '[  0 118   4   2   0   1   0  27   0   4   1]')),
 (8, (20.666, 1.111, '[  0 118   4   2   0   1   0  28   0   4   1]')),
 (9, (23.128, 0.401, '[  0 118   4   2   0   1   0  29   0   4